<a href="https://colab.research.google.com/github/ritwiks9635/My_Neural_Network_Architecture/blob/main/Imbalanced_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Imbalanced classification: credit card fraud detection**

## Introduction

This example looks at the
[Kaggle Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud/)
dataset to demonstrate how
to train a classification model on data with highly imbalanced classes.

In [6]:
import os
os.environ['KAGGLE_USERNAME'] = "keggal_username"
os.environ['KAGGLE_KEY'] = "keggal_user_keys"
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
#os path
api.dataset_download_files( "mlg-ulb/creditcardfraud",path="https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud")
# unzip

In [7]:
!unzip /content/https:/www.kaggle.com/datasets/mlg-ulb/creditcardfraud/creditcardfraud.zip

Archive:  /content/https:/www.kaggle.com/datasets/mlg-ulb/creditcardfraud/creditcardfraud.zip
  inflating: creditcard.csv          


## First, vectorize the CSV data

In [9]:
import csv
import numpy as np

# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
fname = "creditcard.csv"

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


In [10]:
num_val_samples = int(len(features) * 0.2)
train_features = features[: -num_val_samples]
train_targets = targets[: -num_val_samples]

val_features = features[-num_val_samples :]
val_targets = targets[-num_val_samples :]

print("length of training features is :::", len(train_features))
print("length of validation features is :::", len(val_features))

length of training features is ::: 227846
length of validation features is ::: 56961


## Analyze class imbalance in the targets

In [11]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 417 (0.18% of total)


## Normalize the data using training set statistics

In [22]:
mean = np.mean(train_features, axis = 0)
train_features -= mean
val_features -= mean

std = np.std(train_features, axis = 0)
train_features /= std
val_features /= std

In [29]:
import keras

model = keras.Sequential(
    [
        keras.Input(shape = train_features.shape[1:]),
        keras.layers.Dense(256, activation = "relu"),
        keras.layers.Dense(256, activation = "relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation = "relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation = "sigmoid")
    ])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               7936      
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 139777 (546.00 KB)
Trainable params: 13977

## Train the model with `class_weight` argument

In [30]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.keras")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)

Epoch 1/30
112/112 - 9s - loss: 2.4156e-06 - fn: 45.0000 - fp: 28388.0000 - tn: 199041.0000 - tp: 372.0000 - precision: 0.0129 - recall: 0.8921 - val_loss: 0.0658 - val_fn: 11.0000 - val_fp: 391.0000 - val_tn: 56495.0000 - val_tp: 64.0000 - val_precision: 0.1407 - val_recall: 0.8533 - 9s/epoch - 76ms/step
Epoch 2/30
112/112 - 6s - loss: 1.4854e-06 - fn: 38.0000 - fp: 7192.0000 - tn: 220237.0000 - tp: 379.0000 - precision: 0.0501 - recall: 0.9089 - val_loss: 0.1383 - val_fn: 6.0000 - val_fp: 1847.0000 - val_tn: 55039.0000 - val_tp: 69.0000 - val_precision: 0.0360 - val_recall: 0.9200 - 6s/epoch - 56ms/step
Epoch 3/30
112/112 - 6s - loss: 1.2658e-06 - fn: 28.0000 - fp: 6952.0000 - tn: 220477.0000 - tp: 389.0000 - precision: 0.0530 - recall: 0.9329 - val_loss: 0.0877 - val_fn: 9.0000 - val_fp: 1153.0000 - val_tn: 55733.0000 - val_tp: 66.0000 - val_precision: 0.0541 - val_recall: 0.8800 - 6s/epoch - 51ms/step
Epoch 4/30
112/112 - 6s - loss: 1.0451e-06 - fn: 24.0000 - fp: 6970.0000 - tn: 22

## Conclusions

At the end of training, out of 56,961 validation transactions, we are:

- Correctly identifying 66 of them as fraudulent
- Missing 9 fraudulent transactions
- At the cost of incorrectly flagging 441 legitimate transactions

In the real world, one would put an even higher weight on class 1,
so as to reflect that False Negatives are more costly than False Positives.

Next time your credit card gets  declined in an online purchase -- this is why.